# 5. Inference pipeline
This notebook consists of 3 parts:

1. Connecting to the Feature Store and retriving feature views/groups and model
3. Predicting if theres a detection or not
4. Saving the prediction in a new feature view/group

In [1]:
# import libraries
import pandas as pd
import hopsworks
import joblib

## 1. Connecting to the Feature Store and retriving feature views/groups and model

In [2]:
project = hopsworks.login()
fs = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

Multiple projects found. 

	 (1) annikaij
	 (2) miknie20

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/549019
Connected. Call `.close()` to terminate connection gracefully.


In [3]:
# connect to the feature store
project = hopsworks.login()
fs = project.get_feature_store()

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Multiple projects found. 

	 (1) annikaij
	 (2) miknie20
Invalid input, should be an integer from the list of projects.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/549019
Connected. Call `.close()` to terminate connection gracefully.


In [5]:
# Get the bike lane model from the model registry
mr = project.get_model_registry()
bikelane_model = mr.get_model("bikelane_hist_model", version=1)
model_dir = bikelane_model.download()
bikelane_model = joblib.load("/work/Test_MLOps_Project/notebooks/models/bikelane_hist_model.pkl")

Connected. Call `.close()` to terminate connection gracefully.


In [6]:
# Get the building model from the model registry
building_model = mr.get_model("building_hist_model", version=2)
model_dir = building_model.download()
building_model = joblib.load("/work/Test_MLOps_Project/notebooks/models/building_hist_model.pkl")

In [7]:
# Retrieve the feature views for each parking spot
bikelane_feature_view = fs.get_feature_view(name="bikelane_new_fv", version=1)
building_feature_view = fs.get_feature_view(name="building_new_fv", version=1)

In [8]:
# Make predictions on the newest bikelane data
bikelane_data = bikelane_feature_view.get_batch_data()
bikelane_pred = bikelane_model.predict(bikelane_data)

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.70s) 


In [9]:
# Make predictions on the newest building data
building_data = building_feature_view.get_batch_data()
building_pred = building_model.predict(building_data)

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.60s) 


In [10]:
# Add the predictions to the dataframes
bikelane_data['mag_cluster'] = [bikelane_pred[bikelane_pred.size-1]]
building_data['mag_cluster'] = [building_pred[building_pred.size-1]]

In [11]:
#making a column describing if its data from bikelane or building
bikelane_data['data_type'] = ['bikelane']
building_data['data_type'] = ['building']

In [12]:
#combining the two dataframes
data = pd.concat([bikelane_data, building_data])

In [13]:
# upload the predictions to the feature store as a new feature group
latest_pred_fg = fs.get_or_create_feature_group(name="parking_predictions",
                                  version=1,
                                  primary_key=["x", "y", "z"],
                                  description="Predictions for parking spots",
                                  online_enabled=False,
                                 )
latest_pred_fg.insert(data)

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/549019/fs/544841/fg/792929


Uploading Dataframe: 0.00% |          | Rows 0/2 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: parking_predictions_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/549019/jobs/named/parking_predictions_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x7f25ae443e90>, None)